**Further train DeepCell Models with K's Data and Make Predictions**<br>
Starting from model trained with tn nuclear data, further train the model with resized K's images that is cut into 25 512x512 patches, one image at a time.
200 epochs, semanticdatagenerator, resized<br>

In [1]:
starting_model_path = 'tn1.0_nuclear_20221102.h5' # the model trained with nucleus data from the Tissuenet V1.0 dataset
epochs=200 # 20 sec per step 

In [2]:
import syotil

import numpy as np
from skimage import io
from matplotlib import pyplot as plt
%matplotlib inline
from timeit import default_timer
import os
from skimage import img_as_ubyte

import deepcell
from deepcell import image_generators
from deepcell.utils.train_utils import rate_scheduler, get_callbacks, count_gpus
from deepcell_toolbox.utils import resize, tile_image, untile_image
from deepcell_toolbox.deep_watershed import deep_watershed
from deepcell.losses import weighted_categorical_crossentropy
from deepcell.model_zoo.panopticnet import PanopticNet

import tensorflow as tf
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import MSE

print(tf.__version__)
print(deepcell.__version__)

2.7.1
0.11.1


In [3]:
for ii in [1,2,3,4,5,6,7]:
    idx="training{}".format(ii)
    INPUT_PATH="images/"+idx+"/"
    model_name = 'tn1.0_nuclear_K_512x512resized_'+idx 
    model_path = '{}.h5'.format(model_name)

    import glob
    FILENAMES = glob.glob(INPUT_PATH+"*_img.png")
    print(len(FILENAMES))

    imgs = [io.imread(CURR_IM_NAME)[:,:,0] for CURR_IM_NAME in FILENAMES]
    X_train = tf.stack(imgs)
    X_train = np.expand_dims(X_train, axis=-1)
    print(X_train.shape)

    masks = [io.imread(CURR_IM_NAME.replace("img","masks")) for CURR_IM_NAME in FILENAMES]
    y_train = tf.stack(masks)
    y_train = np.expand_dims(y_train, axis=-1)
    print(y_train.shape)

    # resize image
    mpp_select=1.3 # based on DeepCell_tn_nuclear.ipynb, the best value is 1.2, but 1.3 makes scale factor an integer
    shape = X_train.shape
    scale_factor = mpp_select / 0.65
    print(scale_factor)
    new_shape = (int(shape[1] * scale_factor),
                 int(shape[2] * scale_factor))

    X_train_resized = resize(X_train, new_shape, data_format='channels_last')
    y_train_resized = resize(y_train, new_shape, data_format='channels_last', labeled_image=True)
    print(X_train_resized.shape)
    print(y_train_resized.shape)

    # cut each resized image into 5x5 patches of size 512x512 
    print(X_train_resized.shape)
    print(y_train_resized.shape)

    x_starts=[0,400,400*2,400*3,X_train_resized.shape[1]-512]
    x_ends=[x+512 for x in x_starts]
    print(x_starts)
    print(x_ends)

    y_starts=[0,512,512*2,512*3,X_train_resized.shape[2]-512]
    y_ends=[x+512 for x in y_starts]
    print(y_starts)
    print(y_ends)

    sub_X=tf.concat([tf.concat([X_train_resized[:, x_starts[i]:x_ends[i], y_starts[j]:y_ends[j], :] for i in range(len(x_starts)) ], axis=0) for j in range(len(y_starts))], axis=0)
    sub_X = np.array(sub_X, dtype=np.uint8) # tf.stack changes the dtype 
    print(sub_X.shape)
    sub_y=tf.concat([tf.concat([y_train_resized[:, x_starts[i]:x_ends[i], y_starts[j]:y_ends[j], :] for i in range(len(x_starts)) ], axis=0) for j in range(len(y_starts))], axis=0)
    sub_y = np.array(sub_y, dtype=np.uint16) # tf.stack changes the dtype 
    print(sub_y.shape)

    # create dataloader
    seed=0
    min_objects = 5

    transforms = ['inner-distance', 'outer-distance', 'fgbg']
    transforms_kwargs = {'outer-distance': {'erosion_width': 0}}

    # use augmentation for training but not validation
    datagen = image_generators.SemanticDataGenerator(
        rotation_range=180,
        fill_mode='reflect',
        zoom_range=(0.75, 1.25),
        horizontal_flip=True,
        vertical_flip=True)

    datagen_val = image_generators.CroppingDataGenerator()

    batch_size = 4 # 8 causes memory outage

    train_data = datagen.flow(
        {'X': sub_X, 'y': sub_y},
        seed=seed,
        transforms=transforms,
        transforms_kwargs=transforms_kwargs,
        min_objects=min_objects,
        batch_size=batch_size)

    val_data = None

    semantic_classes = [1, 1, 2] # inner distance, outer distance, fgbg

    model = PanopticNet(
        backbone='resnet50',
        input_shape=(512,512,1),
        norm_method='whole_image',
        num_semantic_classes=semantic_classes)

    lr = 1e-4
    optimizer = Adam(lr=lr, clipnorm=0.001)
    lr_sched = rate_scheduler(lr=lr, decay=0.99)

    # Create a dictionary of losses for each semantic head

    def semantic_loss(n_classes):
        def _semantic_loss(y_pred, y_true):
            if n_classes > 1:
                return 0.01 * weighted_categorical_crossentropy(
                    y_pred, y_true, n_classes=n_classes)
            return MSE(y_pred, y_true)
        return _semantic_loss

    loss = {}

    # Give losses for all of the semantic heads
    for layer in model.layers:
        if layer.name.startswith('semantic_'):
            n_classes = layer.output_shape[-1]
            loss[layer.name] = semantic_loss(n_classes)

    model.compile(loss=loss, optimizer=optimizer)

    model.load_weights(starting_model_path, by_name=True)

    [(layer.name, layer.output_shape) for layer in filter(lambda x: x.name.startswith('semantic_'), model.layers)]


    # fit the model
    print('Training on', count_gpus(), 'GPUs.')

    train_callbacks = get_callbacks(
        model_path,
        lr_sched=lr_sched,
    #     monitor="val_loss",
        # monitor='loss', # training loss
        verbose=1)          

    loss_history = model.fit(
        train_data,
        steps_per_epoch=train_data.y.shape[0] // batch_size, # 
        epochs=epochs, 
    #     validation_data=val_data,
    #     validation_steps=val_data.y.shape[0] // batch_size,
        callbacks=train_callbacks)

    model.save_weights(model_path)

    prediction_model = PanopticNet(
        backbone='resnet50',
        norm_method='whole_image',
        num_semantic_classes=[1, 1], # inner distance, outer distance
        input_shape= (512,512,1)
    )

    prediction_model.load_weights(model_path, by_name=True)

    from deepcell.applications import NuclearSegmentation
    app = NuclearSegmentation(prediction_model)
    [(layer.name, layer.output_shape) for layer in filter(lambda x: x.name.startswith('semantic_'), app.model.layers)]


    import os
    print(os.getcwd())
    INPUT_PATH="images/test/"
    FILENAMES = [f for f in os.listdir("images/training/testimages")]
    print(FILENAMES)


    # more detailed mapping of image_mpp parameter
    mpps=[1,1.3]

    AP_mat=[]
    for CURR_IM_NAME in FILENAMES:
        print(CURR_IM_NAME)
        im0 = io.imread(os.path.join(INPUT_PATH, CURR_IM_NAME))
        mask_true=io.imread(os.path.join(INPUT_PATH, CURR_IM_NAME.replace("img","masks")))

        x = np.expand_dims(im0, axis=-1)
        x = np.expand_dims(x, axis=0)
        y, tile_info = app._tile_input(x)
        AP_arr=[]
        for mpp in mpps:
            prd = app._untile_output(app.predict(y, image_mpp=mpp), tile_info)[0,:,:,0]
            a=syotil.csi(mask_true, prd)
            AP_arr.append(a)
        AP_mat.append(AP_arr)

    print(AP_mat)

    import pandas as pd
    df = pd.DataFrame([FILENAMES, [x[0] for x in AP_mat], [x[1] for x in AP_mat]])
    print(df.transpose())
    df.to_csv('images/training/csi_'+model_name+'.txt', index=False, header=False)

1
(1, 1040, 1159, 1)
(1, 1040, 1159, 1)
2.0


2022-11-06 08:49:56.594887: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14657 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:04:00.0, compute capability: 7.0
2022-11-06 08:49:56.618259: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 14657 MB memory:  -> device: 1, name: Tesla V100-PCIE-16GB, pci bus id: 0000:06:00.0, compute capability: 7.0
2022-11-06 08:49:56.620362: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 14657 MB memory:  -> device: 2, name: Tesla V100-PCIE-16GB, pci bus id: 0000:08:00.0, compute capability: 7.0


(1, 2080, 2318, 1)
(1, 2080, 2318, 1)
(1, 2080, 2318, 1)
(1, 2080, 2318, 1)
[0, 400, 800, 1200, 1568]
[512, 912, 1312, 1712, 2080]
[0, 512, 1024, 1536, 1806]
[512, 1024, 1536, 2048, 2318]
(25, 512, 512, 1)
(25, 512, 512, 1)


2022-11-06 08:49:57.442132: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/app/software/TensorFlow/2.7.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training on 3 GPUs.


...

42/42 [==============================] - 24s 561ms/step - loss: 0.0101 - semantic_0_loss: 0.0028 - semantic_1_loss: 0.0062 - semantic_2_loss: 0.0011 - lr: 1.3808e-05
Epoch 199/200
42/42 [==============================] - ETA: 0s - loss: 0.0099 - semantic_0_loss: 0.0028 - semantic_1_loss: 0.0060 - semantic_2_loss: 0.0011WARNING:tensorflow:Can save best model only with val_loss available, skipping.


42/42 [==============================] - 23s 551ms/step - loss: 0.0099 - semantic_0_loss: 0.0028 - semantic_1_loss: 0.0060 - semantic_2_loss: 0.0011 - lr: 1.3670e-05
Epoch 200/200
42/42 [==============================] - ETA: 0s - loss: 0.0101 - semantic_0_loss: 0.0028 - semantic_1_loss: 0.0062 - semantic_2_loss: 0.0011WARNING:tensorflow:Can save best model only with val_loss available, skipping.


42/42 [==============================] - 24s 573ms/step - loss: 0.0101 - semantic_0_loss: 0.0028 - semantic_1_loss: 0.0062 - semantic_2_loss: 0.0011 - lr: 1.3533e-05
/home/yfong/deeplearning/dense_cell_segmentation
['M926910_CFL_Position7_CD3_test_img.png', 'M872956_JML_Position10_CD3_test_img.png', 'M872956_JML_Position8_CD8_test_img.png', 'M926910_CFL_Position13_CD3_test_img.png', 'M872956_JML_Position8_CD3_test_img.png', 'M872956_JML_Position8_CD4_test_img.png', 'M872956_JML_Position9_CD3_test_img.png']
M926910_CFL_Position7_CD3_test_img.png


/app/software/DeepCell/0.11.1-foss-2021b-CUDA-11.4.1/lib/python3.9/site-packages/deepcell_toolbox/deep_watershed.py:179: FutureWarning: `selem` is a deprecated argument name for `h_maxima`. It will be removed in version 1.0.Please use `footprint` instead.
  markers = h_maxima(image=maxima,


M872956_JML_Position10_CD3_test_img.png
M872956_JML_Position8_CD8_test_img.png
M926910_CFL_Position13_CD3_test_img.png
M872956_JML_Position8_CD3_test_img.png
M872956_JML_Position8_CD4_test_img.png
M872956_JML_Position9_CD3_test_img.png
[[0.39028475711892796, 0.4444444444444444], [0.3686131386861314, 0.47007042253521125], [0.5058823529411764, 0.5463917525773195], [0.2945945945945946, 0.37349397590361444], [0.4622467771639042, 0.5590828924162258], [0.5956284153005464, 0.5793269230769231], [0.3577586206896552, 0.44064386317907445]]
                                         0         1         2
0   M926910_CFL_Position7_CD3_test_img.png  0.390285  0.444444
1  M872956_JML_Position10_CD3_test_img.png  0.368613   0.47007
2   M872956_JML_Position8_CD8_test_img.png  0.505882  0.546392
3  M926910_CFL_Position13_CD3_test_img.png  0.294595  0.373494
4   M872956_JML_Position8_CD3_test_img.png  0.462247  0.559083
5   M872956_JML_Position8_CD4_test_img.png  0.595628  0.579327
6   M872956_JML_Position